US Car Sale Data Analysis

This notebook and the related project will be taking a look at a dataset of car sales in the US over the past 20 years, in an effort to determine what parameters of a vehicle inform its price the most. Having this information would ube useful for car manufacturers or dealers to focus on certain vehicle parameters, or for forecastors to make predictions about the actions of those groups.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as stt
import streamlit as st
import matplotlib.pyplot as plt
import plotly.express as px
import altair as alt

In [2]:
cars = pd.read_csv(r"C:\Personal Files\Coding\GitHub\sprint-4-project\vehicles_us.csv", sep=',', decimal='.')
cars.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
49733,6300,2006.0,chevrolet silverado,excellent,8.0,gas,NaN,automatic,truck,grey,1.0,2019-01-03,39
682,12000,2003.0,ford mustang,good,8.0,gas,NaN,manual,coupe,black,NaN,2018-12-09,26
41378,3500,2008.0,honda civic,like new,4.0,gas,142300.0,automatic,sedan,blue,NaN,2018-08-29,50
686,999,1995.0,toyota 4runner,salvage,6.0,gas,164221.0,automatic,SUV,red,NaN,2018-05-03,33
4987,8400,2015.0,hyundai sonata,good,4.0,gas,94688.0,automatic,sedan,silver,NaN,2019-01-16,61
7225,14990,2015.0,toyota rav4,excellent,4.0,gas,19965.0,automatic,SUV,silver,1.0,2018-07-05,30
25957,19800,2014.0,chevrolet silverado,good,8.0,diesel,NaN,automatic,truck,grey,1.0,2019-02-01,55
7394,8749,2014.0,honda civic,like new,4.0,gas,111000.0,automatic,sedan,grey,NaN,2018-11-14,71
37110,4500,2005.0,toyota sienna,good,6.0,gas,191000.0,automatic,mini-van,grey,NaN,2019-03-25,10
2114,36936,2016.0,toyota tundra,excellent,8.0,gas,36320.0,automatic,truck,blue,1.0,2018-08-11,49


In [3]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [4]:
st.header('Car Data')
st.dataframe(cars)

2024-05-09 06:32:50.879 
  command:

    streamlit run c:\Personal Files\Coding\Anaconda3\envs\standard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [5]:
cars.duplicated().sum()

0

In [6]:
cars['model'].value_counts()

model
ford f-150                           2796
chevrolet silverado 1500             2171
ram 1500                             1750
chevrolet silverado                  1271
jeep wrangler                        1119
                                     ... 
ford f-250 super duty                 241
acura tl                              236
kia sorento                           236
nissan murano                         235
mercedes-benz benze sprinter 2500      41
Name: count, Length: 100, dtype: int64

In [7]:
cars['odometer'].value_counts()

odometer
0.0         185
140000.0    183
120000.0    179
130000.0    178
160000.0    167
           ... 
138580.0      1
94624.0       1
123321.0      1
202627.0      1
139573.0      1
Name: count, Length: 17762, dtype: int64

In [8]:
cars['cylinders'].value_counts()

cylinders
8.0     15844
6.0     15700
4.0     13864
10.0      549
5.0       272
3.0        34
12.0        2
Name: count, dtype: int64

In [9]:
cars['is_4wd'].value_counts()

is_4wd
1.0    25572
Name: count, dtype: int64

In [10]:
#The comment lines in this cell block outline how I probably would have tried to sort the missing values without being informed about the transform() method.

#model_year_avg = cars['model_year'].mean()
#cars['model_year'] = cars['model_year'].fillna(model_year_avg)
cars['model_year'] = cars['model_year'].fillna(cars.groupby(['model'])['model_year'].transform('median'))

#mileage_median = cars['odometer'].median()
#cars['odometer] = cars['odometer'].fillna(mileage_median)
cars['odometer'] = cars['odometer'].fillna(cars.groupby(['model'])['odometer'].transform('median'))

#cylinders_mean = cars['cylinders'].mean()
#cars['cylinder'] = cars['cylinder'].fillna(cylinders_mean)
cars['cylinders'] = cars['cylinders'].fillna(cars.groupby(['model'])['cylinders'].transform('mean'))

cars['paint_color'] = cars['paint_color'].fillna('Unknown')

cars['is_4wd'] = cars['is_4wd'].fillna(0.0)

This is a pretty interesting dataset, and there are a number of possibilities for the data analysis that can be done. I would like to compare the prices cars are sold for based on a few different sets of data to get an idea as to how those markers affect the price of a car. In particular, I'd like to explore the relationships the car's price has with its mileage (odometer), the type of car sold, and the length that each car was listed. This could gleam some information as to the conditions under which cars are sold for the highest prices. 

Lots of the ploty.express graphs are very suitable for this with the right data. I will try to do some comparing and contrasting of the infromation conveyed between plots and charts from that library and the ones used for other TripleTen work. As such, some additional grouping will be needed.

In [11]:
mileage = cars.groupby('price')['odometer'].mean().reset_index() #These methods are used to drop the NA values
mileage.sample(5)

,price,odometer
2019,18674,80414.0
1261,11283,149000.0
1750,15905,123044.0
2287,21993,74182.0
2566,25985,130356.0


In [12]:
mileage_price = mileage.groupby('price')
mileage_price
mileage_price.sample()

,price,odometer
0,1,51262.576441
1,3,126684.000000
2,5,140715.000000
3,6,173500.000000
4,9,119.000000
...,...,...
3438,145000,140000.000000
3439,175000,149000.000000
3440,189000,146361.333333
3441,300000,130356.000000


In [13]:
car_type = cars.groupby('type')['price'].mean().reset_index()
car_type.sample(5)

,type,price
3,coupe,14353.442901
12,wagon,9088.134328
11,van,10546.941548
4,hatchback,6868.513849
8,pickup,16057.410418


In [14]:
car_type_price = car_type.groupby('price')
car_type_price.sample()

,type,price
4,hatchback,6868.513849
9,sedan,6965.358647
5,mini-van,8193.177433
12,wagon,9088.134328
11,van,10546.941548
7,other,10989.714844
0,SUV,11149.400000
6,offroad,14292.294393
3,coupe,14353.442901
2,convertible,14575.881166


In [15]:
list_length = cars.groupby('days_listed')['price'].mean().reset_index()
list_length.sample(5)

,days_listed,price
97,97,13523.105882
221,252,11500.000000
89,89,11466.364865
127,127,10086.437500
105,105,11906.716418


In [16]:
list_length_price = list_length.groupby('price')
list_length_price.sample()

,days_listed,price
212,223,1500.0
195,195,1600.5
209,213,1975.5
188,188,2995.0
223,261,3800.0
...,...,...
202,204,25988.0
197,199,27500.0
218,240,31995.0
194,194,34000.0


I initally planned to also compare the differences in plots between pyplot and plotly, but pyplot gave me some issues. So, we'll be focusing on the plotly.express charts, which should still give plenty of information.

In [17]:
st.header('price by mileage')
#cars_json = cars.to_json()
mp_hist = px.histogram(cars, x='price', y='odometer', nbins=120)
st.write(mp_hist)


In [18]:
st.header('price by type')
type_hist = px.histogram(cars, x='price', y='type', nbins=10)
st.write(type_hist)

In [19]:
st.header('price by list time')
time_hist = px.histogram(cars, x='price', y='days_listed', nbins=120)
st.write(time_hist)

In [20]:
st.header('price by year')
year_scatt = px.scatter(cars, x='price', y='days_listed', color='model_year')
st.write(year_scatt)

I've gotten some code that works after a healthy amount of fiddling. There's a lot of code I didn't end up using, and my plans for this project had to change on the fly a bit. Maybe it'll be good practice for the real thing. I will keep the code in for review as a means of showing my prior work.

These charts as represented have some interesting data, though a lot of it is skewed by the low overall prices a majority of the cars were sold for. Price by mileage and price by list time are therefore left with a very strong skew that makes it difficult to judge them with certainty. The Price by year chart show more information and presents a more informative picture - namely, that there doesnt seem to be much of a connection between the price of a car and how long it is sold for. Different vectors of research will be needed to find details which may have correlations between price.

However, the Price by Type graph does offer some very clear information as well. It seems that, at least in terms of price, larger vehicles are doing well in the US market, as the three top selling vehicle types are all larger single family vehicles.